# Loss Functions Part 2

> "In this part of the multi-part series on the loss functions we'll be taking a. look at MSE, MAE, Hinge Loss, Triplet Loss and Huber Loss. We'll also look at the code for these Loss functions in PyTorch and some examples of how to use them"

- toc: true
- branch: fastbook/lessons
- badges: true
- comments: true
- image: images/triplet_loss.png
- categories: [loss_functions]
- hide: false
- author: Akash Mehra

## Mean Squared Error

Mean Squared Error (MSE) measures the average of the squares of the errors for an estimator. The MSE of an estimator $\hat{\theta}$​ wrt an unknown parameter $\theta$​ is defined as $MSE(\hat{\theta})  =E[(\hat{\theta} - \theta)^2]$​​​. It can also be written as the sum of  *Variance* and the squared *Bias* of the estimator. In case of an *unbiased* estimator the *MSE* and *variance* are equivalent. Below is a quick way to see how the MSE and the Variance and Bias are related to each other.

$$
\begin{eqnarray}
E[X] &=& \mu \\\\\\
\end{eqnarray}
$$


$$
\begin{eqnarray}
Var(X) &=& E[(X - \mu)^2] \\\\\\
&=& E[X^2] + \mu^2 - 2\mu E[X] \\\\\\
&=& E[X^2] - E[X]^2
\end{eqnarray}
$$


using the following result:

$$
\begin{eqnarray}
Var(X-Y) &=& Var(X) + Var(Y) - 2Cov(X,Y) \\\\\\
Var(\mu) &=& 0 \\\\\\
Cov(X,\mu) &=& E[X-E[X]]E[\mu-E[\mu]] = 0 \\\\\\
Var(X-\mu) &=& Var(X)
\end{eqnarray}
$$


We arrive at:

$$
\begin{eqnarray}
MSE(\hat{\theta}) &=& E[(\hat{\theta} - \theta)^2] \\\\\\
&=& Var(\hat{\theta} - \theta) + E[\hat{\theta} - \theta]^2 \\\\\\
&=& Var[\hat{\theta}] + Bias^2[\hat{\theta}]
\end{eqnarray}
$$


We can see from above that as *Variance,* *MSE* alwso heavily weights the outliers. It's squaring each term which weights large error terms more heavily than smaller ones. An alternative in that becomes *Mean Absolute Error* which is the topic of discussion of next section.